In [104]:
from dotenv import load_dotenv
load_dotenv()

import os
project_name = "agent_basic"
os.environ["LANGSMITH_PROJECT"] = project_name

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate

### 빌트인 도구
- 랭체인 제공
- 도구와 툴킷
- 툴 -> 단일 도구, 툴킷 -> 여러 도구가 묶여 잇따
- 파일 관리 툴킷 : 파일 삭제 툴, 파일 읽기 툴

### 1. 검색 툴 사용해보기

In [ ]:
# 1. 모델 설정
llm = ChatOpenAI(
    model = "gpt-4.1-mini",
    temperature = 0,
)

# 2. 도구 생성
from langchain_tavily import TavilySearch
tavily_search = TavilySearch()

# 3. 프롬프트 설정
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 툴만 사용해서 사실을 확인하고 한국어로 간단하게 답합니다."),
    ("user", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

# 4. 단일 에이전트 생성
agent = create_openai_tools_agent(
    llm=llm,
    tools=[tavily_search],
    prompt=prompt
)

# 5. excutor 설정
excutor = AgentExecutor(
    agent = agent,
    tools=[tavily_search],
    verbose=True
)

In [10]:
result = excutor.invoke({"input": "서울의 오늘 주요 뉴스 3개 요약해줘"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search` with `{'query': '서울 오늘 주요 뉴스', 'time_range': 'day', 'search_depth': 'basic', 'topic': 'news'}`


{'query': '서울 오늘 주요 뉴스', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.cbsnews.com/newyork/video/first-alert-weather-clearing-skies-for-2nd-half-of-week/', 'title': 'First Alert Weather: Clearing skies for 2nd half of week - CBS News', 'score': 0.15870851, 'published_date': 'Wed, 15 Oct 2025 10:23:00 GMT', 'content': "livelive: CBS News Minnesota### CBS News New York livelive: CBS News New York### CBS News Philadelphia 04:426h ago### President Trump threatens to withhold federal funds if Mamdani wins NYC mayoral election 01:598h ago### First Alert Weather: Cooler temps on the way in New York - 10/14/25 02:339h ago### CC Sabathia named grand marshal of 2025 TCS NYC Marathon 02:2512h ago### New York Public Library exhibit highlights Puerto Ricans in comic books 01:0012h ago 03:1012

In [11]:
result = excutor.invoke({"input": "석촌역 근처 맛집 5개 찾아줘"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search` with `{'query': '석촌역 근처 맛집', 'search_depth': 'basic'}`


{'query': '석촌역 근처 맛집', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.tripadvisor.co.kr/RestaurantsNear-g2024873-d9579729-Seokchon-Seongnam_Gyeonggi_do.html', 'title': '성남 석촌 역 근처 맛집 Best 10 - Tripadvisor - 트립어드바이저', 'content': '석촌 역 근처 맛집 · 1. 미오. 5.0. (74건의 리뷰). 일본식 퓨전$$$$ · 2. 바이킹스워프 롯데월드몰점. 4.5. (219건의 리뷰) · 3. 라티튜드32. 5.0. (91건의 리뷰) · 4.', 'score': 0.9132521, 'raw_content': None}, {'url': 'https://www.diningcode.com/list.dc?query=%EC%84%9D%EC%B4%8C%EC%97%AD%20%EA%B0%80%EC%84%B1%EB%B9%84%EC%A2%8B%EC%9D%80', 'title': '석촌역 가성비좋은 맛집 Top100 - 다이닝코드', 'content': '석촌역 가성비좋은맛집 (111곳) · 1. 단디 송리단길 · 2. 이랑칼국수 석촌역 · 3. 우프 잠실 · 4. 완도산회 포장마차 송파직영점 · 5. 옛촌매운갈비찜 석촌역 · 6. 수작', 'score': 0.8533396, 'raw_content': None}, {'url': 'https://www.tripadvisor.co.kr/RestaurantsNear-g2024873-d9579729-oa60-Seokchon-Seongnam_Gyeonggi_do.html',

### 2. 파일 저장 툴

In [25]:
# 파일 저장 툴 가져오기
from langchain_community.agent_toolkits import FileManagementToolkit

root_dir = "./agent_result"
file_tools = FileManagementToolkit(
    root_dir=root_dir,
).get_tools()

# selected_tools
# CopyFileTool
# DeleteFileTool
# FileSearchTool
# MoveFileTool
# ReadFileTool
# WriteFileTool
# ListDirectoryTool

file_tools

[CopyFileTool(root_dir='./agent_result'),
 DeleteFileTool(root_dir='./agent_result'),
 FileSearchTool(root_dir='./agent_result'),
 MoveFileTool(root_dir='./agent_result'),
 ReadFileTool(root_dir='./agent_result'),
 WriteFileTool(root_dir='./agent_result'),
 ListDirectoryTool(root_dir='./agent_result')]

In [24]:
# 웹서치 툴 가져오기
from langchain_tavily import TavilySearch
tavily_search = TavilySearch(max_result = 5)

In [26]:
# 툴 합치기
total_tool = file_tools + [tavily_search]
total_tool

[CopyFileTool(root_dir='./agent_result'),
 DeleteFileTool(root_dir='./agent_result'),
 FileSearchTool(root_dir='./agent_result'),
 MoveFileTool(root_dir='./agent_result'),
 ReadFileTool(root_dir='./agent_result'),
 WriteFileTool(root_dir='./agent_result'),
 ListDirectoryTool(root_dir='./agent_result'),
 TavilySearch(api_wrapper=TavilySearchAPIWrapper(tavily_api_key=SecretStr('**********'), api_base_url=None))]

In [74]:
# 시스템 프롬프트 설정하기
SYSTEM_PROMPT = """
너는 '파일 시스템 툴'과 '웹검색 툴'만을 사용하는 어시스턴트야

[규칙]
1. 최신 뉴스/링크는 Tavily로 검색한다
2. 사용자가 csv 파일로 저장을 요청하면, 다음 필드 헤더로 csv 를 만든다
    date, query, title, url, source, snippet
3. {YYYYMMDD}.csv 파일이 이미 있는지 확인한다.
4-1. {YYYYMMDD}.csv 파일이 없을때, {YYYYMMDD}.csv 파일을 생성하고 저장한다

4-2. {YYYYMMDD}.csv 파일이 있을 때, 헤더를 유지한채 새로운 데이터를 추가한다
새로운 파일을 생성 금지
덮어쓰기로 기존 내용을 삭제 금지

5. 저장한 뒤에 내용을 확인해서 가장 아래 5줄 정도 정보를 출력한다

모든 답변은 한국어로 간결하게 해라
"""

In [28]:
from datetime import datetime

def get_today():
    return datetime.now().strftime("%Y%m%d")

In [29]:
get_today()

'20251016'

In [73]:
# 1. 모델 설정
llm = ChatOpenAI(
    model = "gpt-4.1-mini",
    temperature = 0,
)

# 2. 도구 생성
from langchain_tavily import TavilySearch
tavily_search = TavilySearch()

# 3. 프롬프트 설정
prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_PROMPT),
    ("user", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

# 4. 단일 에이전트 생성
agent = create_openai_tools_agent(
    llm=llm,
    tools=total_tool,
    prompt=prompt
)

# 5. excutor 설정
exceutor = AgentExecutor(
    agent = agent,
    tools=total_tool,
    verbose=True
)

In [51]:
result = exceutor.invoke({
    "input" : "오늘 AI 관련 뉴스 최신 5개만 요약해주고 그내용을 csv 로 저장해줘",
    "YYYYMMDD" : get_today()
})
print(result)



> Entering new AgentExecutor chain...

Invoking: `tavily_search` with `{'query': 'AI 최신 뉴스', 'search_depth': 'basic', 'time_range': 'day', 'topic': 'news'}`


{'query': 'AI 최신 뉴스', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.mediapost.com/publications/article/409858/huge-acquires-bothyes-creative-studio.html?edition=140212', 'title': 'Huge Acquires Both&Yes Creative Studio 10/14/2025 - MediaPost', 'score': 0.58664185, 'published_date': 'Wed, 15 Oct 2025 11:00:43 GMT', 'content': '* WPP Expands Google AI Partnership, Commits $400M # WPP Expands Google AI Partnership, Commits $400M partnership, which includes a $400 million spending commitment from the holding company for Google\xa0technologies. alongside the company’s increased investment into AI and technology through WPP Open, WPP’s AI platform for marketing. Kurian, "By enabling WPP to innovate on our AI-optimized\xa0technology stack, we are helping create the future of marketing tech

In [48]:
result = exceutor.invoke({
    "input" : "오늘 패션 관련 뉴스 최신 5개만 요약해주고 그내용을 csv 로 저장해줘",
    "YYYYMMDD" : get_today()
})
print(result)



> Entering new AgentExecutor chain...

Invoking: `tavily_search` with `{'query': '패션 최신 뉴스', 'search_depth': 'basic', 'time_range': 'day', 'topic': 'news'}`


{'query': '패션 최신 뉴스', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.businessoffashion.com/news/global-markets/bangladesh-garment-factory-fire-kills-16-toll-may-rise/', 'title': 'Bangladesh Garment Factory Fire Kills 16, Toll May Rise - The Business of Fashion', 'score': 0.528169, 'published_date': 'Wed, 15 Oct 2025 08:20:37 GMT', 'content': 'A fire on Tuesday at a garment factory in Bangladesh and an adjacent chemical warehouse killed at least 16 people and injured others, and the death toll could rise, an official said. “Sixteen bodies have been recovered from the second and third floors of the garment factory,” he said, adding that the number of deaths could rise as recovery operations were continuing. This week’s round-up of global markets fashion business news also features Chin

### 3. 데이터베이스 저장

In [55]:
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain_community.utilities.sql_database import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///./agent_result/agent.db")
db_tools = SQLDatabaseToolkit(db=db, llm=llm).get_tools()
db_tools

[QuerySQLDatabaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x00000299575B6390>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x00000299575B6390>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x00000299575B6390>),
 QuerySQLCheckerTool(description='Use this tool to 

In [56]:
# 툴 합치기
total_tools = db_tools + [tavily_search]
total_tools

[QuerySQLDatabaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x00000299575B6390>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x00000299575B6390>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x00000299575B6390>),
 QuerySQLCheckerTool(description='Use this tool to 

In [87]:
# 시스템 프롬프트 설정하기
SYSTEM_PROMPT = """
## 에이전트 시스템 프롬프트: 웹-DB 연동 관리자

당신은 정보 수집 및 데이터베이스 통합 전문가 에이전트입니다.

### 1. 역할 및 목표 (Role & Goal)
당신의 주된 역할은 사용자 요청에 따라 TavilySearch 툴을 사용하여 웹에서 신뢰성 있는 최신 정보를 검색하고, 그 결과를 SQLDatabaseToolkit 툴을 사용하여 지정된 SQL 데이터베이스에 구조화하여 저장 및 관리하는 것입니다. **테이블 이름은 실행 시점의 날짜 변수 {YYYYMMDD}를 사용하여 'data_{YYYYMMDD}' 형식으로 설정됩니다.** (현재 실행일: data_20251016) 목표는 효율적이고 정확하게 웹 데이터를 DB에 반영하는 것입니다.

### 2. 초기 설정 및 테이블 관리 (Initialization & Table Management)
**파일 이름 및 DB 테이블 명 규칙**: 저장된 결과가 기록될 테이블의 이름은 **'data_{YYYYMMDD}'**입니다. 이 테이블이 존재하지 않는다면, SQLDatabaseToolkit을 사용하여 다음 스키마로 **새롭게 생성**해야 합니다.

CREATE TABLE IF NOT EXISTS data_{YYYYMMDD} (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    date TEXT NOT NULL,
    query TEXT NOT NULL,
    title TEXT,
    url TEXT,
    source TEXT,
    snippet TEXT
);

### 3. 도구 사용 원칙 (Tool Usage Protocol)
1. 정보 검색: 웹에서 정보를 수집해야 할 때만 TavilySearch를 사용합니다.
2. 데이터 관리: 저장, 조회, 수정, 삭제 등 모든 데이터베이스 작업에는 SQLDatabaseToolkit을 사용합니다.
3. SQL 안전성: 생성하는 모든 SQL 쿼리는 DB 스키마에 부합하고, 데이터 무결성을 유지하며, SQL 인젝션 공격에 안전하도록 철저히 검토되어야 합니다.

### 4. 작업 절차 (Execution Workflow)
1. 요청 분석: 사용자 요청이 웹 검색, DB 작업, 또는 두 가지 모두를 필요로 하는지 정확히 판단합니다. 
만약 DB작업이 필요할경우 2,3,4,5 단계를 진행한다.
DB작업이 필요하지 않은 경우 1단계만 진행하고 멈춘다
2. 테이블 확인 및 생성: **SQLDatabaseToolkit을 사용하여 지정된 'data_{YYYYMMDD}' 테이블이 존재하는지 확인하고, 없다면 위 스키마대로 생성합니다.** (이 작업은 사용자 요청과 무관하게 저장 전에 반드시 선행되어야 합니다.)
3. 검색 및 정제: 웹 검색이 필요한 경우, TavilySearch를 실행하여 정보를 수집합니다. 수집된 결과는 테이블 컬럼(date, query, title, url, source, snippet)에 맞게 요약 및 구조화하는 전처리 과정을 거쳐야 합니다.
4. 데이터 저장: 정제된 데이터를 사용하여 SQLDatabaseToolkit을 통해 정확한 **INSERT INTO data_{YYYYMMDD} (...) VALUES (...)** 쿼리를 실행하여 데이터베이스에 반영합니다.
5. 데이터 조회/관리: 조회, 수정, 삭제 요청의 경우, 'data_{YYYYMMDD}' 테이블을 대상으로 적절한 SELECT, UPDATE, DELETE 쿼리를 실행하고 그 결과를 처리합니다.

시작 안내: "요청을 분석하고 있습니다. DB 테이블 'data_{YYYYMMDD}' 설정을 확인하고 웹 검색 및 저장 작업을 시작합니다."
"""

In [88]:
# 1. 모델 설정
llm = ChatOpenAI(
    model = "gpt-4.1-mini",
    temperature = 0,
)

# 2. 도구 생성
from langchain_tavily import TavilySearch
tavily_search = TavilySearch()

# 3. 프롬프트 설정
prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_PROMPT),
    ("user", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

# 4. 단일 에이전트 생성
agent = create_openai_tools_agent(
    llm=llm,
    tools=total_tools,
    prompt=prompt
)

# 5. excutor 설정
exceutor = AgentExecutor(
    agent = agent,
    tools=total_tools,
    verbose=True
)

In [90]:
result = exceutor.invoke({
    "input" : "오늘 AI 관련 뉴스 최신 5개만 요약해주고 그내용을 DB에 저장해줘",
    "YYYYMMDD" : get_today()
})
print(result)



> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with `{}`
responded: 요청을 분석하고 있습니다. DB 테이블 'data_20251016' 설정을 확인하고 웹 검색 및 저장 작업을 시작합니다.

우선, 'data_20251016' 테이블이 존재하는지 확인하고, 없으면 생성하겠습니다.


Invoking: `sql_db_schema` with `{'table_names': 'data_20251016'}`


Error: table_names {'data_20251016'} not found in database
Invoking: `sql_db_query` with `{'query': 'CREATE TABLE IF NOT EXISTS data_20251016 (\n    id INTEGER PRIMARY KEY AUTOINCREMENT,\n    date TEXT NOT NULL,\n    query TEXT NOT NULL,\n    title TEXT,\n    url TEXT,\n    source TEXT,\n    snippet TEXT\n);'}`



Invoking: `tavily_search` with `{'query': 'latest AI news', 'time_range': 'day', 'topic': 'news', 'search_depth': 'advanced', 'include_favicon': False}`


{'query': 'latest AI news', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.forbes.com/sites/petercohan/2025/10/15/ai-bubble-may-pop---wiping-out-40-trillion-learn-what-could-happen-and-what-to-do/', 'ti

---

In [2]:
from langchain_discord.tools.discord_read_messages import DiscordReadMessages
from langchain_discord.tools.discord_send_messages import DiscordSendMessage, DiscordSendMessageSchema

# Create instances of each tool
read_tool = DiscordReadMessages()
send_tool = DiscordSendMessage()

# Read the last 5 messages from channel 1234567890
read_result = read_tool({"channel_id": "1417295078568886326", "limit": 5})
print(read_result)

# Send a message to the same channel
# send_result = send_tool.ainvoke({"channel_id": "1417295078568886326", "message": "Hello from Python!"})
# print(send_result)

Error in discord read messages: asyncio.run() cannot be called from a running event loop


Error reading messages: asyncio.run() cannot be called from a running event loop


c:\Potenup\LLM-Study\10_agent_langchain\langchain_discord\tools\discord_read_messages.py:64: RuntimeWarning: coroutine 'DiscordClientWrapper.read_messages' was never awaited
  return f"Error reading messages: {str(e)}"


In [ ]:
import httpx

response = httpx.post(
    "https://discord.com/api/webhooks/1428271752970108988/09FY4DKOOF2P8GhiMlj4M07YnoguDhMEUSqYAhP7wZufmoogY5VFinveBAMtDP2Ki8jd",
    json={
        "username": "Agent AI",
        "content": "차요준 메롱",
        # you can add embeds here, too
    }
)
# raise an exception if the operation failed.
response.raise_for_status()


<Response [204 No Content]>